In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (roc_auc_score, roc_curve, confusion_matrix,
                             classification_report, f1_score)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
import os
import random
from itertools import combinations
import os
from datetime import datetime

# Répétabilité
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [21]:
# Charger les données prétraitées
df = pd.read_csv("../data/Customers_cleaned.csv")

In [22]:
# Features toujours à supprimer (Churn et customerID)
always_drop = ['Churn', 'customerID']

# Features candidates à supprimer (celles que je veux tester)
features_candidates = [
    'gender', 'PhoneService', 'StreamingMovies', 'StreamingTV',
    'DeviceProtection', 'OnlineBackup', 'MultipleLines'
]

In [23]:
# 4. TEST DE TOUTES LES COMBINAISONS POSSIBLES

# Stockage des résultats
results = []

# Générer toutes les combinaisons possibles de features à supprimer
for r in range(len(features_candidates) + 1):  # +1 pour inclure le cas où aucune feature n'est supprimée
    for combo in combinations(features_candidates, r):
        additional_features_to_drop = list(combo)

        # Toutes les features à supprimer pour cette itération
        features_to_drop = always_drop + additional_features_to_drop

        # Afficher les features supprimées
        if additional_features_to_drop:
            print(f"\n=== Test en supprimant aussi : {additional_features_to_drop} ===")
        else:
            print("\n=== Test avec seulement les features de base supprimées (baseline) ===")

        # Copie du DataFrame pour éviter de modifier l'original
        df_copy = df.copy()

        y = (df_copy['Churn'] == 'Yes').astype(int)
        # Suppression des features
        df_copy.drop(features_to_drop, axis=1, inplace=True)

        # Séparation features/target

        X = df_copy

        # Définition des colonnes
        numeric_features = [col for col in X.columns if X[col].dtype != 'object']
        categorical_features = [col for col in X.columns if X[col].dtype == 'object']

        # Création du preprocesseur
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numeric_features),
                ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
            ] if categorical_features else [
                ('num', StandardScaler(), numeric_features)
            ]
        )

        # Split des données (train/val/test stratifié)
        X_temp, X_test, y_temp, y_test = train_test_split(
            X, y, test_size=0.2, random_state=SEED, stratify=y
        )
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=0.2, random_state=SEED, stratify=y_temp
        )

        # Fit/transform sur train, transform sur val/test
        X_train_prep = preprocessor.fit_transform(X_train)
        X_val_prep = preprocessor.transform(X_val)
        X_test_prep = preprocessor.transform(X_test)


=== Test avec seulement les features de base supprimées (baseline) ===

=== Test en supprimant aussi : ['gender'] ===

=== Test en supprimant aussi : ['PhoneService'] ===

=== Test en supprimant aussi : ['StreamingMovies'] ===

=== Test en supprimant aussi : ['StreamingTV'] ===

=== Test en supprimant aussi : ['DeviceProtection'] ===

=== Test en supprimant aussi : ['OnlineBackup'] ===

=== Test en supprimant aussi : ['MultipleLines'] ===

=== Test en supprimant aussi : ['gender', 'PhoneService'] ===

=== Test en supprimant aussi : ['gender', 'StreamingMovies'] ===

=== Test en supprimant aussi : ['gender', 'StreamingTV'] ===

=== Test en supprimant aussi : ['gender', 'DeviceProtection'] ===

=== Test en supprimant aussi : ['gender', 'OnlineBackup'] ===

=== Test en supprimant aussi : ['gender', 'MultipleLines'] ===

=== Test en supprimant aussi : ['PhoneService', 'StreamingMovies'] ===

=== Test en supprimant aussi : ['PhoneService', 'StreamingTV'] ===

=== Test en supprimant aussi :

In [24]:
# 5. CRÉATION ET ENTRAÎNEMENT DU MODÈLE
# Définition des hyperparamètres
input_dim = X_train_prep.shape[1]
lr = 0.001
dropout = 0.3

# Création du modèle
inputs = layers.Input(shape=(input_dim,), name='tabular_input')
x = layers.Dense(128, activation='relu')(inputs)
x = layers.Dropout(dropout)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(dropout)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs, name='telco_mlp')
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr),
    loss='binary_crossentropy',
    metrics=[
        keras.metrics.AUC(name='auc'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.Precision(name='precision')
    ]
)

model.summary()


Model: "telco_mlp"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tabular_input (InputLayer)      │ (None, 18)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,753 (42.00 KB)

 Trainable params: 10,753 (42.00 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
#Entrainement du MLP
#Création des répertoires pour les checkpoints et les logs
checkpoints_dir= "checkpoints"
tensorboard_dir= "tensorboard_logs"
os.makedirs(checkpoints_dir, exist_ok= True)
os.makedirs(tensorboard_dir, exist_ok= True)

In [26]:
# Création des dossiers pour les artefacts si besoin
run_id = f"drop_{'_'.join(additional_features_to_drop) if additional_features_to_drop else 'aucune'}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
checkpoints_dir = os.path.join("checkpoints", run_id)
tensorboard_dir = os.path.join("tensorboard_logs", run_id)
visual_dir = os.path.join("../visualisations", run_id)
os.makedirs(checkpoints_dir, exist_ok=True)
os.makedirs(tensorboard_dir, exist_ok=True)
os.makedirs(visual_dir, exist_ok=True)

# Callbacks
cb_early_stopping = callbacks.EarlyStopping(
    monitor='val_auc',
    patience=10,  # Réduit pour accélérer les tests
    restore_best_weights=True,
    mode='max'
)

cb_checkpoint= callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoints_dir, 'best_model.keras'),
    monitor= "val_auc",
    save_best_only= True,
    mode= 'max'
)

cb_tensorboard= callbacks.TensorBoard(
    log_dir= tensorboard_dir,
    histogram_freq=1
)


#Gestion du déséquilibre de classes
neg, pos = np.bincount(y_train)
class_weight= {0: 1.0, 1: neg/pos}




# Entraînement du modèle
batch_size = 32
history = model.fit(
    X_train_prep, y_train,
    validation_data=(X_val_prep, y_val),
    epochs=30,  # Réduit pour accélérer les tests
    batch_size=batch_size,
    callbacks=[cb_early_stopping, cb_checkpoint, cb_tensorboard],
    class_weight= class_weight ,
    verbose=0  # Désactiver le verbose pour la clarté
)

In [27]:
print("NaN dans X_train_prep :", np.isnan(X_train_prep).sum())
print("NaN dans X_val_prep :", np.isnan(X_val_prep).sum())
print("Inf dans X_train_prep :", np.isinf(X_train_prep).sum())
print("Inf dans X_val_prep :", np.isinf(X_val_prep).sum())

NaN dans X_train_prep : 0
NaN dans X_val_prep : 0
Inf dans X_train_prep : 0
Inf dans X_val_prep : 0


In [28]:
# Prédictions sur l'ensemble de validation
y_val_pred_proba = model.predict(X_val_prep, verbose=0).ravel()  # Aplatir les prédictions

# Vérifier et gérer les valeurs NaN
if np.isnan(y_val_pred_proba).any():
    print(f"Attention : {np.isnan(y_val_pred_proba).sum()} valeurs NaN détectées dans les prédictions.")
    # Remplacer les NaN par 0.5 (valeur neutre)
    y_val_pred_proba = np.nan_to_num(y_val_pred_proba, nan=0.5)

# Convertir en prédictions binaires
y_val_pred = (y_val_pred_proba > 0.5).astype(int)

# Calcul des métriques
val_auc = roc_auc_score(y_val, y_val_pred_proba)
val_f1 = f1_score(y_val, y_val_pred)

# Affichage des métriques
print("Performances du modèle MLP sur l'ensemble de validation :")
print(f"ROC AUC : {val_auc:.4f}")
print(f"F1 Score : {val_f1:.4f}")
print("\nRapport de classification :")
print(classification_report(y_val, y_val_pred))

# Stockage des résultats
results.append({
    'additional_features_dropped': ', '.join(additional_features_to_drop) if additional_features_to_drop else 'Aucune',
    'num_additional_dropped': len(additional_features_to_drop),
    'ROC_AUC': val_auc,
    'F1': val_f1
})

Performances du modèle MLP sur l'ensemble de validation :
ROC AUC : 0.8481
F1 Score : 0.6199

Rapport de classification :
              precision    recall  f1-score   support

           0       0.91      0.71      0.80       828
           1       0.50      0.81      0.62       299

    accuracy                           0.74      1127
   macro avg       0.71      0.76      0.71      1127
weighted avg       0.80      0.74      0.75      1127



In [33]:
# Matrice de confusion
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_val, y_val_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Non Churn', 'Churn'],
            yticklabels=['Non Churn', 'Churn'])
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de confusion - MLP')
plt.tight_layout()
plt.savefig(os.path.join(visual_dir, "matrice_confusion.png"))
plt.close()

In [37]:
# Courbe ROC
plt.figure(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_val, y_val_pred_proba)
plt.plot(fpr, tpr, label=f'AUC = {val_auc:.4f}')
plt.plot([0, 1], [0, 1], 'k--', label='Aléatoire')
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbe ROC - MLP')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(visual_dir, "roc_curve.png"))
plt.close()

In [31]:
# Distribution des scores de prédiction
plt.figure(figsize=(10, 6))
sns.histplot(y_val_pred_proba[y_val == 0], bins=50, alpha=0.5, label='Non Churn', color='blue')
sns.histplot(y_val_pred_proba[y_val == 1], bins=50, alpha=0.5, label='Churn', color='red')
plt.axvline(x=0.5, color='black', linestyle='--', label='Seuil (0.5)')
plt.xlabel('Score de prédiction')
plt.ylabel('Nombre d\'exemples')
plt.title('Distribution des scores de prédiction par classe')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(visual_dir, "distribution_scores.png"))
plt.close()

In [34]:
# 7. ANALYSE DES RÉSULTATS
# Conversion en DataFrame
results_df = pd.DataFrame(results)

# Tri par ROC AUC décroissant
results_df = results_df.sort_values('ROC_AUC', ascending=False)

# Affichage des résultats
print("\n=== Résultats triés par ROC AUC ===")
display(results_df)

# Sauvegarde des résultats
results_df.to_csv("../data/feature_combinations_results.csv", index=False)
print("\nRésultats sauvegardés dans feature_combinations_results.csv")


=== Résultats triés par ROC AUC ===


,additional_features_dropped,num_additional_dropped,ROC_AUC,F1
0,"gender, PhoneService, StreamingMovies, Streami...",7,0.848077,0.619898



Résultats sauvegardés dans feature_combinations_results.csv
